In [1]:
%matplotlib inline

# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/claud/Class/allhw/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Lebu,CL,-37.6167,-73.6500,54.46,96,100,1.14,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,62.11,88,75,11.50,broken clouds
2,2,Kandava,LV,57.0409,22.7747,42.76,85,5,4.85,clear sky
3,3,Hilo,US,19.7297,-155.0900,87.28,77,100,9.22,overcast clouds
4,4,Padilla,CO,3.2204,-76.3139,81.97,90,69,2.13,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & 
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
3,3,Hilo,US,19.7297,-155.0900,87.28,77,100,9.22,overcast clouds
4,4,Padilla,CO,3.2204,-76.3139,81.97,90,69,2.13,broken clouds
5,5,Georgetown,MY,5.4112,100.3354,80.55,89,20,2.30,few clouds
7,7,Vaini,TO,-21.2000,-175.2000,82.56,78,20,4.61,few clouds
11,11,Takoradi,GH,4.8845,-1.7554,77.97,87,100,6.82,overcast clouds
13,13,Worland,US,44.0169,-107.9554,73.85,13,0,16.11,clear sky
14,14,Kapaa,US,22.0752,-159.3190,80.58,81,40,18.41,scattered clouds
18,18,Bud,US,39.4470,-86.1758,70.83,60,69,2.91,broken clouds
19,19,East London,ZA,-33.0153,27.9116,73.58,48,85,5.03,overcast clouds
20,20,Wajid,SO,3.8093,43.2471,78.01,72,98,14.41,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 3 to 687
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City_ID          273 non-null    int64  
 1   City             273 non-null    object 
 2   Country          272 non-null    object 
 3   Lat              273 non-null    float64
 4   Lng              273 non-null    float64
 5   Max Temp         273 non-null    float64
 6   Humidity         273 non-null    int64  
 7   Cloudiness       273 non-null    int64  
 8   Wind Speed       273 non-null    float64
 9   Current Weather  273 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.5+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.clean_preferred_cities_df= preferred_cities_df.dropna()
clean_df = preferred_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 3 to 687
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City_ID          272 non-null    int64  
 1   City             272 non-null    object 
 2   Country          272 non-null    object 
 3   Lat              272 non-null    float64
 4   Lng              272 non-null    float64
 5   Max Temp         272 non-null    float64
 6   Humidity         272 non-null    int64  
 7   Cloudiness       272 non-null    int64  
 8   Wind Speed       272 non-null    float64
 9   Current Weather  272 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.4+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,
4,Padilla,CO,81.97,broken clouds,3.2204,-76.3139,
5,Georgetown,MY,80.55,few clouds,5.4112,100.3354,
7,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
11,Takoradi,GH,77.97,overcast clouds,4.8845,-1.7554,
13,Worland,US,73.85,clear sky,44.0169,-107.9554,
14,Kapaa,US,80.58,scattered clouds,22.0752,-159.3190,
18,Bud,US,70.83,broken clouds,39.4470,-86.1758,
19,East London,ZA,73.58,overcast clouds,-33.0153,27.9116,
20,Wajid,SO,78.01,overcast clouds,3.8093,43.2471,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params['location'] = f'{lat},{lng}'
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... Skipping.')

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 3 to 687
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             272 non-null    object 
 1   Country          272 non-null    object 
 2   Max Temp         272 non-null    float64
 3   Current Weather  272 non-null    object 
 4   Lat              272 non-null    float64
 5   Lng              272 non-null    float64
 6   Hotel Name       272 non-null    object 
dtypes: float64(3), object(4)
memory usage: 25.1+ KB


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# Syntax to replace empty string,with NaN and then drop the NaN
# Https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_df.dropna(subset=['Hotel Name'], inplace = True)
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257 entries, 3 to 685
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             257 non-null    object 
 1   Country          257 non-null    object 
 2   Max Temp         257 non-null    float64
 3   Current Weather  257 non-null    object 
 4   Lat              257 non-null    float64
 5   Lng              257 non-null    float64
 6   Hotel Name       257 non-null    object 
dtypes: float64(3), object(4)
memory usage: 16.1+ KB


In [11]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
3,Hilo,US,87.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Padilla,CO,81.97,broken clouds,3.2204,-76.3139,La Granadina
5,Georgetown,MY,80.55,few clouds,5.4112,100.3354,Cititel Penang
7,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Takoradi,GH,77.97,overcast clouds,4.8845,-1.7554,Raybow International Hotel
...,...,...,...,...,...,...,...
681,Tautira,PF,78.22,light rain,-17.7333,-149.1500,Pension Oaoa
682,Port-Gentil,GA,80.56,scattered clouds,-0.7193,8.7815,Fengshui Residence
683,Bull Savanna,JM,77.61,scattered clouds,17.8868,-77.5902,Big Red's
684,Humberto De Campos,BR,75.92,scattered clouds,-2.5983,-43.4611,Pousada do Cabeça


In [12]:
# 8a. Create the output File (CSV)
output_data_file = '/Users/claud/Class/allhw/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp,
                                dissipating = False, max_intensity = 300, point_radius = 4)
fig.add_layer(heat_layer)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))